In [ ]:
"""
core/router.py - The Dynamic LLM Router.

This module implements the `HeuristicRouter` class, which dynamically selects
the most appropriate LLM model for a given query based on a simple heuristic
and user feedback.
"""

class HeuristicRouter:
    """
    Dynamically routes user queries to different LLM models.

    The routing decision is based on the length of the query and a feedback-based
    bias. This enables a simple form of dynamic adaptation, where the model
    can "learn" to use a more powerful (and potentially more expensive) model
    when it receives negative feedback.
    """
    def __init__(self):
        """Initializes the router with a default bias."""
        # This bias increases with negative feedback, pushing the router to
        # use the more powerful Claude 3.5 Sonnet model.
        self.bias_towards_35 = 0

    def select_model(self, query: str) -> str:
        """
        Selects an LLM model based on a heuristic.

        Args:
            query: The user's question.

        Returns:
            The name of the selected LLM model.
        """
        # If the query is long (>50 characters) or the bias is high (>2),
        # use the more powerful Claude 3.5 Sonnet.
        if len(query) > 50 or self.bias_towards_35 > 2:
            return "claude-3-5-sonnet-20240620"
        return "claude-3-sonnet-20240229"

    def adjust_routing(self, thumbs_up: bool):
        """
        Adjusts the routing bias based on user feedback.

        Args:
            thumbs_up: A boolean indicating positive (True) or negative (False) feedback.
        """
        if not thumbs_up:
            self.bias_towards_35 += 1
        else:
            # On positive feedback, reduce the bias but never below zero
            self.bias_towards_35 = max(0, self.bias_towards_35 - 1)